<a href="https://colab.research.google.com/github/leonardofalango/files/blob/main/latex_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# Install dependencies

import typing
import collections

# Donwload Files to test
!cd /content
!rm -rf test_data
!mkdir test_data
!cd test_data && wget https://raw.githubusercontent.com/leonardofalango/files/refs/heads/main/latex_parser/val1.txt
!cd test_data && wget https://raw.githubusercontent.com/leonardofalango/files/refs/heads/main/latex_parser/val2.txt
!cd test_data && wget https://raw.githubusercontent.com/leonardofalango/files/refs/heads/main/latex_parser/val3.txt

--2025-04-23 17:04:55--  https://raw.githubusercontent.com/leonardofalango/files/refs/heads/main/latex_parser/val1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59 [text/plain]
Saving to: ‘val1.txt’

val1.txt            100%[===================>]      59  --.-KB/s    in 0s      

2025-04-23 17:04:56 (1.05 MB/s) - ‘val1.txt’ saved [59/59]

--2025-04-23 17:04:56--  https://raw.githubusercontent.com/leonardofalango/files/refs/heads/main/latex_parser/val2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45 [text/plain]
Saving

In [55]:
# Check if files were downloaded
!ls test_data

val1.txt  val2.txt  val3.txt


# Lexer

In [140]:
import re

# Tokens
TOKEN_REGEX = [
  ('CONSTANTE', r'\btrue\b|\bfalse\b'),
  ('OPERADORUNARIO', r'\\neg'),
  ('OPERADORBINARIO', r'\\wedge|\\vee|\\rightarrow|\\leftrightarrow'),
  ('ABREPAREN', r'\('),
  ('FECHAPAREN', r'\)'),
  ('PROPOSICAO', r'\b[0-9][0-9a-z]*\b'),
  ('ESPACO', r'\s+'),
]

class Token:
  def __init__(self, tipo : str, value : typing.Any):
    self.tipo = tipo
    self.value = value

  # For printing
  def __repr__(self):
    return f'Token({self.tipo}, {repr(self.value)})'

def lexer(exp : str):
  tokens = []
  pos = 0
  while pos < len(exp):
    match = None
    for token_type, regex in TOKEN_REGEX:
      pattern = re.compile(regex)
      match = pattern.match(exp, pos)
      if match:
        if token_type != 'ESPACO':
            valor = match.group(0)
            tokens.append(Token(token_type, valor))
        pos = match.end()
        break
    if not match:
      raise SyntaxError(f'Token inválido na posição {pos}: {exp[pos]}')
  return tokens


In [141]:
# Testing lexer
exp = r"(\wedge 1x true)"
tokens = lexer(exp)
for t in tokens:
    print(t)

Token(ABREPAREN, '(')
Token(OPERADORBINARIO, '\\wedge')
Token(PROPOSICAO, '1x')
Token(CONSTANTE, 'true')
Token(FECHAPAREN, ')')


# Parser

In [36]:
class Parser:
  def __init__(self, tokens : typing.List[Token]):
    self.tokens = tokens # Tokens by lexer
    self.pos = 0
    self.current : Token = self.tokens[self.pos] if self.tokens else None

  # Next token
  def advance(self):
      self.pos += 1
      self.current = self.tokens[self.pos] if self.pos < len(self.tokens) else None

  # Match token with token_type (tipo)
  def match(self, expected_type : str):
    if self.current and self.current.tipo == expected_type:
        self.advance()
    else:
        raise SyntaxError(f'Esperado {expected_type}, mas encontrou {self.current}')

  # Parse method to expr
  def parse(self):
    self.formula()
    if self.current is not None:
        raise SyntaxError(f'Tokens restantes: {self.current}')

  # FORMULA = CONSTANTE | PROPOSICAO | FORMULAUNARIA | FORMULABINARIA
  def formula(self):
    if self.current.tipo in ['CONSTANTE', 'PROPOSICAO']:
        self.advance()
    elif self.current.tipo == 'ABREPAREN':
        self.advance()
        if self.current.tipo == 'OPERADORUNARIO':
            self.formula_unaria()
        elif self.current.tipo == 'OPERADORBINARIO':
            self.formula_binaria()
        else:
            raise SyntaxError(f'Esperado operador após "(", encontrou: {self.current}')
    else:
        raise SyntaxError(f'Expressão mal formada, encontrou: {self.current}')

  # FORMULAUNARIA = ABREPAREN OPERADORUNARIO FORMULA FECHAPAREN
  def formula_unaria(self):
    self.match('OPERADORUNARIO')
    self.formula()
    self.match('FECHAPAREN')

  # FORMULABINARIA = ABREPAREN OPERADORBINARIO FORMULA FORMULA FECHAPAREN
  def formula_binaria(self):
    self.match('OPERADORBINARIO')
    self.formula()
    self.formula()
    self.match('FECHAPAREN')

In [124]:
# Testing

def is_valid(expr: str):
  try:
      tokens = lexer(expr) # Getting tokens
      parser = Parser(tokens)
      parser.parse() # Validating tokens
      return True
  except SyntaxError:
      return False

# Formating the ouput
def validate(expr: str):
  return "válida" if is_valid(expr) else "inválida"

exp = r"(\wedge 1x true)"
print(is_valid(exp))
exp = r"(\wedge (1x true)"
print(is_valid(exp))

True
False


# Testing

In [101]:
# Foreach testing file, foreach line use validate(line)
def parse_file(filepath: str):
  # Open and read the file
  with open(filepath, "r") as f:
    # verify if the file is valid
    n_formulas = int(f.readline())
    print(f"Arquivo: {filepath} Quantidade de validações: {n_formulas}")

    # read all lines
    lines = f.readlines()
    if n_formulas != len(lines):
      raise Exception("Arquivo contendo conteudo inválido, quantidade de validações não bate com a quantidade de linhas")

    # foreach formula validade
    for i in range(n_formulas):
      line = lines[i].strip()
      p = validate(line)
      print(f"[{p:<{8}}] Expressão: {line}")

In [114]:
# Test file 1
filepath = "test_data/val1.txt"
parse_file(filepath=filepath)

Arquivo: test_data/val1.txt Quantidade de validações: 4
[válida  ] Expressão: (\neg true)
[inválida] Expressão: (\wedge a1 3b)
[inválida] Expressão: (\neg)
[válida  ] Expressão: (\vee (\neg 0x) false)


In [103]:
# Test file 2
filepath = "test_data/val2.txt"
parse_file(filepath=filepath)

Arquivo: test_data/val2.txt Quantidade de validações: 3
[válida  ] Expressão: (\neg (\wedge true false))
[inválida] Expressão: (p1)
[válida  ] Expressão: (\neg true)


In [113]:
# Test file 3
filepath = "test_data/val3.txt"
parse_file(filepath=filepath)

Arquivo: test_data/val3.txt Quantidade de validações: 8
[inválida] Expressão: (\rightarrow (\neg false) (\wedge 1a a2))
[inválida] Expressão: (\leftrightarrow (\vee a1 b2) (\neg 9z))
[válida  ] Expressão: (\wedge (\neg (\neg true)) (\vee 0x false))
[inválida] Expressão: (\leftrightarrow (\neg (\wedge true false)) (\vee x9 y8))
[inválida] Expressão: (\rightarrow (\vee p1) q1)
[válida  ] Expressão: (\leftrightarrow true false)
[inválida] Expressão: (\wedge (\neg true) (\vee))
[inválida] Expressão: (\vee (\vee a1 b1))


$(\neg (\wedge (\vee (\rightarrow 1a 1b) (\wedge true 2a)) 3z))$

In [146]:
is_valid(r'(\neg (\wedge (\vee (\rightarrow 1a 1b) (\wedge true 2a)) 3z))')

True